In [43]:
import duckdb

In [44]:
db = duckdb.connect('./data/data.db')

### Create Base Tables From CSVs

In [45]:
db.sql(
"""
CREATE OR REPLACE TABLE nfl_betting AS
SELECT 
    regexp_extract(filename, 'nfl_betting_(\\d{4})\\.csv', 1)::INT AS Year,
    * EXCLUDE(filename)
FROM read_csv('./data/nfl_betting_*', union_by_name = true, filename = true);
"""
)

In [46]:
db.sql(
"""
CREATE OR REPLACE TABLE nfl_rankings AS
SELECT 
    regexp_extract(filename, 'nfl_rankings_(\\d{4})\\.csv', 1)::INT AS Year,
    * EXCLUDE(filename)
FROM read_csv('./data/nfl_rankings_*', union_by_name = true, filename = true);
"""
)

In [ ]:
# db.sql(
# """
# -- Update all records for the Rams franchise to 'LA Rams'
# UPDATE nfl_betting
# SET Team = 'LA Rams'
# WHERE Team IN ('St Louis', 'Los Angeles');

# -- Update all records for the Chargers franchise to 'LA Chargers'
# UPDATE nfl_betting
# SET Team = 'LA Chargers'
# WHERE Team = 'San Diego';

# -- Update all records for the Raiders franchise to 'Las Vegas Raiders'
# UPDATE nfl_betting
# SET Team = 'Las Vegas Raiders'
# WHERE Team IN ('Oakland', 'Las Vegas');

# -- Update all records for the Rams franchise to 'LA Rams'
# UPDATE nfl_rankings
# SET Team = 'LA Rams'
# WHERE Team IN ('St Louis', 'Los Angeles');

# -- Update all records for the Chargers franchise to 'LA Chargers'
# UPDATE nfl_rankings
# SET Team = 'LA Chargers'
# WHERE Team = 'San Diego';

# -- Update all records for the Raiders franchise to 'Las Vegas Raiders'
# UPDATE nfl_rankings
# SET Team = 'Las Vegas Raiders'
# WHERE Team IN ('Oakland', 'Las Vegas');
# """
# )

### Create Feature Tables

In [47]:
db.sql(
"""
SELECT 
    Year,
    Week,
    Team,
    Division,
    Rating,
    ROW_NUMBER() OVER (PARTITION BY Year, Week ORDER BY Rating DESC) AS Rank
FROM nfl_rankings
"""
).df()

,Year,Week,Team,Division,Rating,Rank
0,2016,15,New England,AFC East,7.1,1
1,2016,15,Atlanta,NFC South,4.8,2
2,2016,15,Pittsburgh,AFC North,4.7,3
3,2016,15,Dallas,NFC East,4.4,4
4,2016,15,Kansas City,AFC West,4.3,5
...,...,...,...,...,...,...
6907,2024,13,Tennessee,AFC South,-5.0,28
6908,2024,13,New England,AFC East,-7.5,29
6909,2024,13,NY Giants,NFC East,-7.6,30
6910,2024,13,Las Vegas,AFC West,-7.6,31


In [51]:
db.sql(
"""
CREATE OR REPLACE TABLE game_features AS
WITH nfl_betting_base AS (
    SELECT 
        Year,
        Week,
        Date,
        Team,
        Opponent,
        Location,
        Spread,
        "Score Team" AS Score_Team,
        "Score Opponent" AS Score_Opponent,
        Won
    FROM nfl_betting
    WHERE Year >= 2013
),

nfl_rankings_base AS (
    SELECT 
        Year,
        Week,
        Team,
        Division,
        Rating,
        ROW_NUMBER() OVER (PARTITION BY Year, Week ORDER BY Rating DESC) AS Rank
    FROM nfl_rankings
    WHERE Year >= 2013
),

games AS (
    SELECT
        t1.Year,
        t2.Week,
        t1.Date,
        CASE 
            WHEN t1.Location = 'Home' THEN t1.Team
            WHEN t1.Location = 'Away' THEN t1.Opponent
            ELSE t1.Team
        END AS Home_Team,
        CASE 
            WHEN t1.Location = 'Away' THEN t1.Team
            WHEN t1.Location = 'Home' THEN t1.Opponent
            ELSE t1.Opponent
        END AS Away_Team
    FROM nfl_betting_base t1
    JOIN nfl_betting_base t2
        ON 
            t1.Year = t2.Year AND t1.Week = t2.Week
            AND t1.Team = t2.Opponent AND t1.Opponent = t2.Team
            AND t1.Team < t1.Opponent -- avoid duplicate games
),

team_features AS (
    SELECT
        *,
        IFNULL(
            DATE_DIFF('day', LAG(Date) OVER (PARTITION BY Year, Team ORDER BY Week ASC), Date),
        14) AS Days_Since_Last_Game,
        COUNT(*) OVER (PARTITION BY Year, Team ORDER BY Week ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS Games_Played,
        IFNULL(
            SUM(Won) OVER (PARTITION BY Year, Team ORDER BY Week ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING),
        0) AS Wins,
        IFNULL(
            COUNTIF(Won = 0) OVER (PARTITION BY Year, Team ORDER BY Week ASC ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING),
        0) AS Losses,

    FROM nfl_betting_base 
),

team_features_with_rank AS (
    SELECT
        tf.*,
        nr.Rank
    FROM team_features as tf
    INNER JOIN nfl_rankings_base as nr
        ON tf.Year = nr.Year AND tf.Week = nr.Week AND tf.Team = nr.Team
)

SELECT
    g.*,
    IF(home_t.Location = 'Neutral', 1, 0) AS Is_Neutral,
    home_t.Spread,
    home_t.Rank AS Home_Rank,
    away_t.Rank AS Away_Rank,
    home_t.Days_Since_Last_Game AS Home_Days_Since_Last_Game,
    away_t.Days_Since_Last_Game AS Away_Days_Since_Last_Game,
    home_t.Games_Played AS Home_Games_Played,
    away_t.Games_Played AS Away_Games_Played,
    home_t.Wins AS Home_Wins,
    away_t.Wins AS Away_Wins,
    home_t.Losses AS Home_Losses,
    away_t.Losses AS Away_Losses,
    
    home_t.Score_Team AS Home_Score,
    home_t.Score_Opponent AS Away_Score,
    home_t.Won AS Home_Won,
FROM games as g
JOIN team_features_with_rank as home_t
    ON g.Year = home_t.Year AND g.Week = home_t.Week
        AND g.Home_Team = home_t.Team AND home_t.Location IN ('Home', 'Neutral')
JOIN team_features_with_rank as away_t
    ON g.Year = away_t.Year AND g.Week = away_t.Week
        AND g.Away_Team = away_t.Team AND away_t.Location IN ('Away', 'Neutral')
ORDER BY g.Year, g.Week, g.Home_Team, g.Away_Team
;
"""
)#.df()

##### QA, Need to Rename some teams

In [34]:
db.sql(
"""
SELECT 
    Team,
    COUNT(DISTINCT Year) as Years,
    ARRAY_AGG(DISTINCT Year ORDER BY Year ASC) AS Years_List
FROM nfl_betting
GROUP BY ALL
HAVING Years < 25
ORDER BY Years ASC
"""
).df()

,Team,Years,Years_List
0,Houston,23,"[2002, 2003, 2004, 2005, 2006, 2007, 2008, 200..."


In [42]:
db.sql(
"""
SELECT 
   Year, COUNT(*) as Games_Count
FROM game_features
GROUP BY ALL
ORDER BY Year
"""
).df()

,Year,Games_Count
0,2013,210
1,2014,212
2,2015,210
3,2016,210
4,2017,240
5,2018,240
6,2019,240
7,2020,240
8,2021,255
9,2022,254
